In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import datetime
import czmlizer
import importlib
importlib.reload(czmlizer)

In [ ]:
df = pd.read_csv("training.csv")
pd.set_option('display.max_columns', None)

# Remove Participants 1 and 37 because of corrupt data

df = df.loc[df['PARTICIPANT_KEY'] != 1.0]
df = df.loc[df['PARTICIPANT_KEY'] != 37.0]


# Add the UTC_DATETIME column
def combine_utc_date_and_time(utc_date, utc_time):
    return datetime.datetime.combine(datetime.datetime.strptime(utc_date, '%Y-%m-%d %H:%M:%S').date(), datetime.datetime.strptime(utc_time, '%H:%M:%S').time())

df['UTC_DATETIME'] = df.apply(lambda row: combine_utc_date_and_time(row['UTC_DATE'], row['UTC_TIME']), axis=1)

In [ ]:
def add_session_ids(data, gap):
    session_id = 0 
    participant = 0.0
    previous = None
    result = []
    for index, row in data.iterrows():
        if row['PARTICIPANT_KEY'] != participant: # must be someone new
            participant = row['PARTICIPANT_KEY']
            session_id += 1
        elif row['UTC_DATETIME'] < previous: # out of order data, must be new
            session_id += 1
        elif row['UTC_DATETIME'] - previous > gap: # big gap, must be new
            session_id += 1
        previous = row['UTC_DATETIME']
        result.append(session_id)
    data['SESSION_ID'] = result

In [ ]:
def add_mission_ids(interval, df):
    df = df.sort_values(by=['UTC_DATETIME'])
    mission_id = -1
    for unique_row in df['IHC'].unique():
        ihc = df[df['IHC'] == unique_row]
        smallest_index = int(ihc.index.min())
        last_time_step = ihc.loc[smallest_index, 'UTC_DATETIME']
        mission_id = mission_id + 1
        for (index, row) in ihc.iterrows():
            if (row['UTC_DATETIME'] - last_time_step) > interval:
                mission_id = mission_id + 1
                last_time_step = row['UTC_DATETIME']
            df.at[index, 'MISSION_ID'] = mission_id     

In [ ]:
interval = pd.Timedelta(hours=6)
add_mission_ids(interval, df)

In [ ]:
no_missions = df['MISSION_ID'].max()
for mission in range(no_missions):
    mission_data = df[df['MISSION_ID'] == mission]
    mission_idx = df[df['MISSION_ID'] == mission].index.min()
    ihc = df.loc[mission_idx,'IHC']
    datetime_start = df.loc[mission_idx,"UTC_DATETIME"]
    cz = czmlizer.Builder()
    for participant in mission_data['PARTICIPANT_KEY'].unique():
        participant_data = mission_data[mission_data['PARTICIPANT_KEY'] == participant]
        trajectory = list(zip(list(participant_data['UTC_DATETIME']), 0.0 - participant_data['LONGITUDE'], participant_data['LATITUDE']))
        cz.add_ground_entity(participant, "../Cesium-1.104/Apps/SampleData/models/ArrowMarker/ArrowMarker.glb", trajectory, f"Participant {participant}", minimum_pixel_size = 25)
    fname = f"./czml_missions/{mission}_{ihc}_{datetime_start}.czml".replace(" ", "")
    with open(fname, 'w') as file:
        file.write(cz.dumps())